# Setup

## Load packages


In [ ]:
using CSV
using DataFrames
using DataFramesMeta
using Distributions
using Plots
using StatsPlots
using Unitful

Plots.default(; margin=5Plots.mm)

## Local package


In [ ]:
using Revise
using HouseElevation

# Build the House Object

The code below helps us build an object to hold important parameters, including the house's description, occupancy, and source.  We also include the area of the house, the height above the gauge, and the value of the house.  We then use the House() constructor to build the house object.


In [ ]:
haz_fl_dept = CSV.read("data/haz_fl_dept.csv", DataFrame)

In [ ]:
#| output: false
house = let
    haz_fl_dept = CSV.read("data/haz_fl_dept.csv", DataFrame) 
    desc = "two story, no basement, Structure"
    occ = "RES1"
    src = "USACE - Galveston"
    row = @rsubset(haz_fl_dept, :Description == desc, :Occupancy == occ, :Source == src)[1, :] 
    area = 881u"ft^2"
    height_above_gauge = 4*u"ft" # the House() constructor could not input a float, so I had to round 4.3 down to 4 so that it would be an int type instead
    value_usd = 194_456
    House(
        row;
        area=area,
        height_above_gauge=height_above_gauge,
        value_usd=value_usd,
    )
end

The code below creates the depth-damages function used to determine how much cost can be expected at a given flood depth.


In [ ]:
let
    depths = uconvert.(u"ft", (-7.0u"ft"):(1.0u"inch"):(30.0u"ft"))
    damages = house.ddf.(depths) .* house.value_usd ./ 1000
    scatter(
        depths,
        damages;
        xlabel="Flood Depth",
        ylabel="Damage (Thousand USD)",
        label="$(house.description)\n($(house.source))",
        legend=:bottomright,
        size=(800, 400),
        yformatter=:plain, # prevents scientific notation
    )
end

The code below establishes the cost of raising a house to a given elevation.  This is important as it will help us determine the cost of raising the house to a given elevation.


In [ ]:
let
    elevations = 0u"ft":0.25u"ft":14u"ft"
    costs = [elevation_cost(house, eᵢ) for eᵢ in elevations]
    scatter(
        elevations,
        costs ./ 1_000;
        xlabel="Elevation",
        ylabel="Cost (Thousand USD)",
        label="$(house.description)\n($(house.source))",
        legend=:bottomright,
        size=(800, 400),
        yformatter=:plain, # prevents scientific notation
    )
end

# Read in SLR


In [ ]:
#| output: false
slr_scenarios = let
    df = CSV.read("data/slr_oddo.csv", DataFrame)
    [Oddo17SLR(a, b, c, tstar, cstar) for (a, b, c, tstar, cstar) in eachrow(df)]
end
println("There are $(length(slr_scenarios)) parameter sets");

We can plot these scenarios to get a sense of the range of sea-level rise we might expect.


In [ ]:
let
    years = 1900:2150
    p = plot(;
        xlabel="Year",
        ylabel="Mean sea-level (ft)\nwith respect to the year 2000",
        label="Oddo et al. (2017)",
        legend=false
    )
    for s in rand(slr_scenarios, 250)
        plot!(p, years, s.(years); color=:lightgrey, alpha=0.5, linewidth=0.5)
    end
    p
end

# Draw Samples from Storm Surge and Discount rate

We can draw samples from the storm surge and discount rate distributions to get a sense of the range of values we might expect.


In [ ]:
function draw_surge_distribution()
    μ = rand(Normal(5, 1))
    σ = rand(Exponential(1.5))
    ξ = rand(Normal(0.1, 0.05))
    GeneralizedExtremeValue(μ, σ, ξ)
end

We can then call this function many times to get many different distributions for the storm surge.
For example,


In [ ]:
[draw_surge_distribution() for _ in 1:1000]

The code below establishes a function that makes this random draw for us.  This will be called later as we run the different combinations.


In [ ]:
#| output: false
function draw_discount_rate()
    draw = rand(Normal(0.04, 0.02))
    if draw < 0
        return 0.001
    else
        return draw
    end
end

# Run Simulations

## One Simulation

The code below demonstrates, for only one combination, how we can run the simultion on given SOW, model, and action.


In [ ]:
p = ModelParams(
    house=house,
    years=2024:2083
)

sow = SOW(
    rand(slr_scenarios),
    draw_surge_distribution(),
    draw_discount_rate()
)

a = Action(3.0u"ft")

res = run_sim(a, sow, p)

The code below shows a very early analysis of these results.  The simulation is run 100 times on different combinations, and the NPV is plotted.  The 25th, 50th, and 75th quantiles are also plotted.  This is a very preliminary analysis, but it is interesting to see the range of NPVs that we might expect.


In [ ]:
restrack=[];
itrack = [];
for i = 1:100
    sow = SOW(
        rand(slr_scenarios),
        draw_surge_distribution(),
        draw_discount_rate()
    )
    res = run_sim(a, sow, p)
    push!(restrack, res)
    push!(itrack, i)
end

q25 = quantile(restrack, 0.25)
q75 = quantile(restrack, 0.75)
q50 = quantile(restrack, 0.50)
# plot restrack and quantiles
plot(itrack, restrack, label="Simulated Results", xlabel="Iteration", ylabel="Cost (USD)", legend=:bottomright)
hline!([q25], label="25th Quantile", color=:red)
hline!([q50], label="50th Quantile", color=:black)
hline!([q75], label="75th Quantile", color=:red)
annotate!(25, q50-100000, text("Mean = $q50", :left, 10, :black))
plot!(title="Simulated Results")
# shade in the vlaues of y between q25 and q75

## Multiple Simulations

Below, I create a vector of possible action choices (e.g. house elevations).  This allows me to randomly sample from the actions like I am randomly selecting from the other SOWs.


In [ ]:
action_dist = [0, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5]
sows = [SOW(rand(slr_scenarios), draw_surge_distribution(), draw_discount_rate()) for _ in 1:500] # for 10 SOWs
actions = [Action(rand(action_dist)*u"ft") for _ in 1:500] # these are all the same
results = [run_sim(a, s, p) for (a, s) in zip(actions, sows)]

The code below creates a dataframe with the results of the simulation.  This will allow us to analyze the results in a more organized way.


In [ ]:
df = DataFrame(
    npv=results,
    Δh_ft=[a.Δh_ft for a in actions],
    slr_a=[s.slr.a for s in sows],
    slr_b=[s.slr.b for s in sows],
    slr_c=[s.slr.c for s in sows],
    slr_tstar=[s.slr.tstar for s in sows],
    slr_cstar=[s.slr.cstar for s in sows],
    surge_μ=[s.surge_dist.μ for s in sows],
    surge_σ=[s.surge_dist.σ for s in sows],
    surge_ξ=[s.surge_dist.ξ for s in sows],
    discount_rate=[s.discount_rate for s in sows],
)

# Analysis

Now that we have our dataframe with our different SOWs and action combinations, and their associated NPVs, we can analyze both what the best NPV scenarios are, and how the different variables impacted our results.

First, we can use the code below to sort our dataframe by NPV and find the best 10 scenarios.


In [ ]:
df = sort(df, :npv)
best10df = last(df,10)

Based on this sorted dataframe, we can see that, within our 500 combinations, the lowest NPV is -93,801.  The other top 10 scenarios are similar.  However, it is interesting to note that there is still sufficient variability in the other variables that produced the similar Top 10 scenarios.

For example, the discount rate for these top 10 ranges between 0.0049 and 0.0807. The plots below show the how the different parameters vary between these 10 scenarios.


In [ ]:
## PLOT THE STORM SURGE

plot()
for i = 1:10
    surge_dist = GeneralizedExtremeValue(best10df.surge_μ[i], best10df.surge_σ[i], best10df.surge_ξ[i])
    global plotdist = plot!(surge_dist; title = "Surge from Top 10", xlims=(2,7), ylims=(0,15))
end
display(plotdist)

## PLOT THE DISCOUNT RATE

plotdiscount = plot()
discount_rates = best10df.discount_rate  
x_values = [1,2,3,4,5,6,7,8,9,10]  
plotdiscount = scatter(x_values, discount_rates; title = 
    "Discount Rate from Top 10", xlabel="Scenario", ylabel="Discount Rate", leg=false, ylims=(0,0.15))

## PLOT THE HEIGHT 

plotheight = plot() 
heights = best10df.Δh_ft
x_values = [1,2,3,4,5,6,7,8,9,10]
plotheight = scatter(x_values, heights; title = "Height from 
    Top 10", xlabel="Scenario", ylabel="Height (ft)", leg=false, ylims=(0,15))

## COMBINED Plots

plotend = plot(plotheight, plotdist, plotdiscount, layout=(2,2))

The results provided in these very preliminary graphs are interesting.  The height, though there is variation, seems concentrated around 5 within our top scenarios.  This seems, at first glance, logical as it represents a relatively high elevation but does not elevate enough to be incredibly expensive in terms of original construction.

The discount rate has a large range from 0.0049 to 0.0807.  The plots below show the huge difference caused by the changes in discount rate.  Notice that, for the scenario in which the discount rate is 0.08, money is almost worthless by the year 2060 and would not be considered. 


In [ ]:
x = collect(2024:2083)  
y1 = 1 ./ ((1 + 0.0049) .^ (x .- 2023))  
y2 = 1 ./ ((1 + 0.0807) .^ (x .- 2023)) 
plot(x, y1, label="Discount Rate = 0.0049", xlabel="Year", 
    ylabel="Discounted Value", title="Discount Rate Impact", legend=:bottomright)
plot!(x, y2, label="Discount Rate = 0.0807")

It is clear, then, that discount rates have a huge impact on the NPV of the project.  With a higher discount rate, we might only consider the near immediate future; hypothetically, if this were extreme, people might want to avoid raising their homes at all.  This allows them to not spend money in the near future, and they do not care about the risk of flood damages later because the money is not worth as much then.

This is a very interesting phenomena, then, that the discount rate is so important in determining the "worth" or "reward" of our actions but the top scenarios have such a large range.

The surge also shows an interesting behavior.  I have copied the surge plot here. There is a large range in the location parameter going from 3.02 to 6.0.  However, these storm surge values, though there is a range, are relatively low.  This is completely expected as lower flood levels mean less potential for damages.  It is interesting as two of the storm surges show a behavior in which they almost look like a spike.  This means that the storm surge is very likely to be at a certain level and unlikely to fall anywhere else.  In both cases, the expected and very likely storm surge is just below the house elevation, which would mean that the house would not be flooded but only the absolutely necessary construction was done.


In [ ]:
plot()
for i = 1:10
    surge_dist = GeneralizedExtremeValue(best10df.surge_μ[i], best10df.surge_σ[i], best10df.surge_ξ[i])
    global plotdist = plot!(surge_dist; title = "Surge from Top 10", xlims=(2,7), ylims=(0,15))
end
display(plotdist)

Note that, as our dataframe stores the variables which determine sea level rise, I did not plot those specifically.  However, it is clear that sea-level-rise is similarly important in determining these damages.

In terms of where we get the best results, it is difficult to identify any singular nswer in that performance within these different categories is inter-dependent.  For example, if the height is just above a very concentrated storm surge distribution, we benefit from just raising the house to reap the benefits without paying more than we needed to.  Discount rates strongly impact whether we take action now or risk it later.  Sea-level rise is similarly important as it shifts our distributions upwards, producing more risk every year.

For this reason, (in my opinion) to say where we get the best results, we would need to consider all of these factors together, an artifact of the complexity of these variables, their uncertainties, and their interdependencies.

I would make a similar argument for the worst 10 results.  I have plotted a dataframe with the worst 10 below.  In this particular run, we see the NPV ranges from 13,680,000 to 2,387,000.  


In [ ]:
df = sort(df, :npv)
worst10df = first(df,10)

Interestingly, we see individual parameters that are relatively similar to parameters in our best scenarios.  The heights can range around 5, the location parameter seems to range from 3 to 7, and the discount rate showss great variation (though this might be slightly lower).  Clearly parameters in isolation cannot produce either a good or a bad scenario.  It is the specific combinations that determine whether we are rewarded for our actions.

As for the computing power, I would try to run more scenarios.  Given my personal conclusion that it is the combinations of parameters, not the parameters themselves, which determine the utility of a choice, it is essential to observe as many combinations as possible.  I was limited, due to my own computer, to only 500 combinations.  With uncertainties in many of the inputs, many runs are required.

Finally, regarding the implications of these results for decision making, I think this lab is a great example of exploratory modeling.   It is not completely deterministic in the sense that there is no right answer.  There are different correct answers dependent on the situation, but understanding these possibilities is useful and gives insight that allows us to make the best choice we can.

# Other graphs

I have included other exploratory plots and figures I made below in case they are of interest!


In [ ]:
# plot df Δh_ft vs df npv
scatter(df.Δh_ft, df.npv, xlabel="Δh (ft)", ylabel="NPV (USD)", legend=false)

In [ ]:
surge_dist_2 = GeneralizedExtremeValue(4, 1.5, 0.1)
action_dist = [0, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5]
sows = [SOW(rand(slr_scenarios), surge_dist_2, draw_discount_rate()) for _ in 1:500] # for 10 SOWs
actions = [Action(rand(action_dist)*u"ft") for _ in 1:500] # these are all the same
results = [run_sim(a, s, p) for (a, s) in zip(actions, sows)]

In [ ]:
df = DataFrame(
    npv=results,
    Δh_ft=[a.Δh_ft for a in actions],
    slr_a=[s.slr.a for s in sows],
    slr_b=[s.slr.b for s in sows],
    slr_c=[s.slr.c for s in sows],
    slr_tstar=[s.slr.tstar for s in sows],
    slr_cstar=[s.slr.cstar for s in sows],
    surge_μ=[s.surge_dist.μ for s in sows],
    surge_σ=[s.surge_dist.σ for s in sows],
    surge_ξ=[s.surge_dist.ξ for s in sows],
    discount_rate=[s.discount_rate for s in sows],
)

In [ ]:
# plot df Δh_ft vs df npv

scat2 = scatter(df.Δh_ft, df.npv, xlabel="Δh (ft)", ylabel="NPV (USD)", legend=false)
# plot the distribution surge_dist_2
plot2 = plot(surge_dist_2; label="Surge Distribution", color=:red, xlims=(-5,25), ylims=(0,0.3))
subplot2 = plot(scat2, plot2, layout=(1, 2))

In [ ]:
surge_dist_3 = GeneralizedExtremeValue(7, 1.5, 0.1)
action_dist = [0, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5]
sows = [SOW(rand(slr_scenarios), surge_dist_3, draw_discount_rate()) for _ in 1:500] # for 10 SOWs
actions = [Action(rand(action_dist)*u"ft") for _ in 1:500] # these are all the same
results = [run_sim(a, s, p) for (a, s) in zip(actions, sows)]

In [ ]:
df = DataFrame(
    npv=results,
    Δh_ft=[a.Δh_ft for a in actions],
    slr_a=[s.slr.a for s in sows],
    slr_b=[s.slr.b for s in sows],
    slr_c=[s.slr.c for s in sows],
    slr_tstar=[s.slr.tstar for s in sows],
    slr_cstar=[s.slr.cstar for s in sows],
    surge_μ=[s.surge_dist.μ for s in sows],
    surge_σ=[s.surge_dist.σ for s in sows],
    surge_ξ=[s.surge_dist.ξ for s in sows],
    discount_rate=[s.discount_rate for s in sows],
)

In [ ]:
# plot df Δh_ft vs df npv

scat3 = scatter(df.Δh_ft, df.npv, xlabel="Δh (ft)", ylabel="NPV (USD)", legend=false)
# plot the distribution surge_dist_2
plot3 = plot(surge_dist_3; label="Surge Distribution", color=:red, xlims=(-5,25), ylims=(0,0.3))
subplot3 = plot(scat3, plot3, layout=(1, 2))

In [ ]:
surge_dist_4 = GeneralizedExtremeValue(15, 1.5, 0.1)
action_dist = [0, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5]
sows = [SOW(rand(slr_scenarios), surge_dist_3, draw_discount_rate()) for _ in 1:500] # for 10 SOWs
actions = [Action(rand(action_dist)*u"ft") for _ in 1:500] # these are all the same
results = [run_sim(a, s, p) for (a, s) in zip(actions, sows)]

In [ ]:
df = DataFrame(
    npv=results,
    Δh_ft=[a.Δh_ft for a in actions],
    slr_a=[s.slr.a for s in sows],
    slr_b=[s.slr.b for s in sows],
    slr_c=[s.slr.c for s in sows],
    slr_tstar=[s.slr.tstar for s in sows],
    slr_cstar=[s.slr.cstar for s in sows],
    surge_μ=[s.surge_dist.μ for s in sows],
    surge_σ=[s.surge_dist.σ for s in sows],
    surge_ξ=[s.surge_dist.ξ for s in sows],
    discount_rate=[s.discount_rate for s in sows],
)

In [ ]:
# plot df Δh_ft vs df npv

scat4 = scatter(df.Δh_ft, df.npv, xlabel="Δh (ft)", ylabel="NPV (USD)", legend=false)
# plot the distribution surge_dist_2
plot4 = plot(surge_dist_4; label="Surge Distribution", color=:red, xlims=(-5,25), ylims=(0,0.3))
subplot4 = plot(scat4, plot4, layout=(1, 2))

In [ ]:
subplotend = plot(subplot2, subplot4, layout=(2,1))